In [ ]:
# Code for recommending dishes to new users
# Input Food DB json; User Interaction Log json
# Output a popularity based recommendation for new users

In [35]:
import pandas as pd 
import numpy as np

# Importing db of food items across all canteens registered on the platform
df = pd.read_json('Food DB.json')

In [36]:
# determine popularity based on user interactions log
user_interaction_count = pd.read_json('User interaction Log.json')
user_interaction_count['Score'] = user_interaction_count['Searched'] + user_interaction_count['Purchased'] + user_interaction_count['Liked'] 
user_interaction_count = user_interaction_count.groupby('Name')['Score'].sum().reset_index()

In [37]:
# add the popularity column to the recepies_data df
df = pd.merge(df,user_interaction_count, how='left', left_on='Name', right_on='Name')
df['Score'].fillna(0, inplace=True)

In [38]:
# Average Reviews of all items
C= df['Review'].mean()

# Min votes required to appear in recommendation list fixed at 60% among 'Score'
m= df['Score'].quantile(0.6)

# shortlist dishes above 60%ile votes
q_items = df.copy().loc[df['Score'] >= m]

# Calculation of weighted rating based on the IMDB formula
def weighted_rating(x, m=m, C=C):
    v = x['Score']
    R = x['Review']
    return (v/(v+m) * R) + (m/(m+v) * C)

# Applying weighted_rating to qualified items
q_items['weighted_score'] = q_items.apply(weighted_rating, axis=1)

# Shortlisting the top rated items and popular items
top_rated_items = q_items.sort_values('weighted_score', ascending=False)
pop_items= df.sort_values('Score', ascending=False)

In [39]:
# Display results of demographic filtering
top_rated_items[['Name', 'Review', 'Score']].head()

,Name,Review,Score
5541,White Chocolate Pistachio Bars Recipe | White ...,5.000000,6.0
764,Bathua Raita Recipe,5.000000,5.0
1685,Chicken Souvlaki with Tzatziki Recipe,4.980392,8.0
5902,Whole Wheat Methi Garlic Naan Recipe | Oven M...,4.979198,8.0
314,Homemade Tagliatelle Pasta with Capers and Bel...,4.994135,5.0


In [40]:
pop_items[['Name','Review', 'Score']].head()

,Name,Review,Score
6094,Doi Bhetki Recipe - Bengali Style Yogurt Fish ...,4.882413,14.0
826,Beetroot Carrot Tomato & Ginger Juice Recipe,4.809942,13.0
1472,Carrot and Oats Whole Wheat Bread Recipe,4.893175,12.0
5118,Chicken Keema Paratha Recipe - Minced Chicken ...,4.944489,12.0
6356,Kerala Style Mutton Pepper Fry Recipe,4.883259,11.0
